Cho đến nay, chúng ta đã thảo luận về cách xử lý dữ liệu và cách xây dựng, huấn luyện và kiểm thử các mô hình học sâu. Tuy nhiên, đến một lúc nào đó, hy vọng chúng ta sẽ hài lòng với các mô hình đã học được và muốn lưu lại kết quả để sử dụng sau này trong nhiều ngữ cảnh khác nhau (có thể thậm chí để đưa ra dự đoán trong quá trình triển khai). Thêm vào đó, khi thực hiện một quá trình huấn luyện dài, phương pháp tốt nhất là định kỳ lưu lại các kết quả trung gian (checkpointing) để đảm bảo rằng chúng ta không bị mất dữ liệu tính toán trong vài ngày nếu chẳng may làm hỏng dây nguồn của máy chủ. Do đó, đã đến lúc tìm hiểu cách tải và lưu trữ cả các vectơ trọng số riêng lẻ và toàn bộ mô hình. Phần này sẽ giải quyết cả hai vấn đề đó.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
#đối với các tensor, dùng save và load
x = torch.arange(4)
torch.save(x, 'x-file')

In [3]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [4]:
# lưu danh sách các tensor\
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [5]:
#đọc và viết 1 từ điển ánh xạ từ chuỗi sang tensor
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [6]:
# Lưu tham số của mô hình
#chỉ lưu trữ tham số của mô hình, không lưu trữ toàn bộ mô hình
# -> cần phải định nghĩa lại mô hình khi tải tham số
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [7]:
# lưu trữ các tham số của mô hình vào một tệp có tên “mlp.params”.
torch.save(net.state_dict(), 'mlp.params')

In [8]:
#khoi phục mô hình
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [9]:
#cùng tham số -> kqua giống nhau
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])